In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df = pd.read_csv('gemstone.csv')
df=df.drop(columns=['id'])

ordinal_mapping = {
    'cut': ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],
    'color': ['D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'clarity': ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
}
encoder = OrdinalEncoder(categories=[ordinal_mapping['cut'],
                         ordinal_mapping['color'],
                         ordinal_mapping['clarity']], dtype=int)
df[['cut', 'color', 'clarity']] = encoder.fit_transform(df[['cut', 'color', 'clarity']])

original_features = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']
X = df[original_features]
y = np.log(df['price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

results = []
current_features = original_features.copy()
current_X_train = X_train.copy()
current_X_test = X_test.copy()

In [2]:
scaler = StandardScaler().fit(current_X_train)
X_scaled = scaler.transform(current_X_train)

model = LinearRegression()
baseline_rmse = -cross_val_score(model, X_scaled, y_train,
                               cv=5, scoring='neg_root_mean_squared_error').mean()
results.append(('Baseline', baseline_rmse, current_features.copy()))

print(f"Baseline RMSE: {baseline_rmse:.4f}")

Baseline RMSE: 0.1395


In [3]:
current_X_train = current_X_train.assign(
    volume = current_X_train.x * current_X_train.y * current_X_train.z,
    log_carat = np.log(current_X_train.carat),
    depth_table_ratio = current_X_train.depth / current_X_train.table
)

current_X_test = current_X_test.assign(
    volume = current_X_test.x * current_X_test.y * current_X_test.z,
    log_carat = np.log(current_X_test.carat),
    depth_table_ratio = current_X_test.depth / current_X_test.table
)

new_features = ['volume', 'log_carat', 'depth_table_ratio']
current_features += new_features

scaler = StandardScaler().fit(current_X_train)
X_scaled = scaler.transform(current_X_train)

model = LinearRegression()
rmse = -cross_val_score(model, X_scaled, y_train,
                      cv=5, scoring='neg_root_mean_squared_error').mean()
results.append(('Basic Transformations', rmse, new_features))

In [4]:
print(f"After Basic Transformations RMSE: {rmse:.4f}")
print(f"Improvement: {baseline_rmse - rmse:.4f}")
print("Added features:", new_features)

After Basic Transformations RMSE: 0.1369
Improvement: 0.0026
Added features: ['volume', 'log_carat', 'depth_table_ratio']


In [5]:
current_X_train['carat_bin'] = pd.qcut(current_X_train.carat, q=5, labels=False, duplicates='drop')
current_X_train['depth_bin'] = pd.qcut(current_X_train.depth, q=5, labels=False, duplicates='drop')

current_X_test['carat_bin'] = pd.qcut(current_X_test.carat, q=5, labels=False, duplicates='drop')
current_X_test['depth_bin'] = pd.qcut(current_X_test.depth, q=5, labels=False, duplicates='drop')

new_features = ['carat_bin', 'depth_bin']
current_features += new_features


scaler = StandardScaler().fit(current_X_train)
X_scaled = scaler.transform(current_X_train)

model = LinearRegression()
rmse = -cross_val_score(model, X_scaled, y_train,
                      cv=5, scoring='neg_root_mean_squared_error').mean()
results.append(('Binning', rmse, new_features))

In [6]:
print(f"After Binning RMSE: {rmse:.4f}")
print(f"Improvement: {baseline_rmse - rmse:.4f}")
print("Added features:", new_features)

After Binning RMSE: 0.1369
Improvement: 0.0027
Added features: ['carat_bin', 'depth_bin']


In [7]:
current_X_train['carat_volume'] = current_X_train.carat * current_X_train.volume
current_X_train['table_depth'] = current_X_train.table * current_X_train.depth

current_X_test['carat_volume'] = current_X_test.carat * current_X_test.volume
current_X_test['table_depth'] = current_X_test.table * current_X_test.depth

new_features = ['carat_volume', 'table_depth']
current_features += new_features

scaler = StandardScaler().fit(current_X_train)
X_scaled = scaler.transform(current_X_train)

model = LinearRegression()
rmse = -cross_val_score(model, X_scaled, y_train,
                      cv=5, scoring='neg_root_mean_squared_error').mean()
results.append(('Interaction Features', rmse, new_features))

In [8]:
print(f"After Interaction Features RMSE: {rmse:.4f}")
print(f"Improvement: {baseline_rmse - rmse:.4f}")
print("Added features:", new_features)

After Interaction Features RMSE: 0.1302
Improvement: 0.0094
Added features: ['carat_volume', 'table_depth']


In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
    'n_estimators': [10, 20]
}

param_dist = {
    'n_estimators': [5,10,20],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 15]
}

rf = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search.fit(current_X_train, y_train)
print(f"Best Grid Search RMSE: {-grid_search.best_score_:.2f}")

random_search = RandomizedSearchCV(rf, param_dist, n_iter=3, cv=5,
                                   scoring='neg_root_mean_squared_error', random_state=42, n_jobs=-1)
random_search.fit(current_X_train, y_train)
print(f"Best Random Search RMSE: {-random_search.best_score_:.2f}")

Best Grid Search RMSE: 0.11
Best Random Search RMSE: 0.11


In [10]:
print("Best Parameters from Grid Search:")
print(grid_search.best_params_)

print("\nBest Parameters from Random Search:")
print(random_search.best_params_)

Best Parameters from Grid Search:
{'n_estimators': 20}

Best Parameters from Random Search:
{'n_estimators': 20, 'min_samples_split': 5, 'max_depth': None}
